In [70]:
from pyvista import imred, tv, spectra
import pyvista.data
from importlib_resources import files
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

In [71]:
%matplotlib qt
t=tv.TV()

In [72]:
red=imred.Reducer('KOSMOS',dir='/home/holtz/raw/apo/jul22/UT220709')

INSTRUMENT: KOSMOS   config: 
  will use format:  /home/holtz/raw/apo/jul22/UT220709/*{:04d}.f*.fits*
         gain:  [0.6]    rn: [ 5.]
         scale:  None   
  Biastype : 1
  Bias box: 
    SC    NC    SR    NR
  2055    43    20  4056 
  2105    43    20  4056 
  Trim box: 
    SC    NC    SR    NR
     0  2048     0  4096 
     0  2048     0  4096 
  Norm box: 
    SC    NC    SR    NR
  1000    51  2000    51 


In [85]:
red.log()

FILE,DATE-OBS,OBJNAME,RA,DEC,EXPTIME
bytes42,bytes26,bytes23,bytes16,bytes16,bytes16
MOS2_quartz_flat_image.0001.fits,2022-07-09T03:21:07.134252,,6:56:00.00,70:00:00.00,0.5
MOS2_quartz_flat_image.0002.fits,2022-07-09T03:24:45.689872,,6:56:00.00,70:00:00.00,0.5
MOS2_quartz_flat_image.0003.fits,2022-07-09T03:28:26.731187,,6:56:00.00,70:00:00.00,0.5
bias.0001.fits,2022-07-09T06:23:41.763063,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0002.fits,2022-07-09T06:24:35.610926,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0003.fits,2022-07-09T06:25:29.285990,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0004.fits,2022-07-09T06:26:23.050087,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0005.fits,2022-07-09T06:27:17.175363,2MASS 1746,17:46:11.99,50:34:03.60,0
bias.0006.fits,2022-07-09T06:28:11.720817,2MASS 1746,17:46:11.99,50:34:03.60,0


In [73]:
he=red.rd('He_truss_blue.0002.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/He_truss_blue.0002.fits


In [6]:
t.tv(he)

In [8]:
intarc=red.rd('NeKrAr_b.0001.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/NeKrAr_b.0001.fits


In [9]:
t.tv(intarc)

In [11]:
obj=red.rd('SDSSJ222655.88p255009.3_r.0012.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/SDSSJ222655.88p255009.3_r.0012.fits


In [12]:
t.tv(obj)

In [13]:
field=red.rd('field_image_no-mask_11985_rot90.0001.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/field_image_no-mask_11985_rot90.0001.fits


In [14]:
t.tv(field)

In [74]:
mask=red.rd('field_image_mask_11985.0001.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/field_image_mask_11985.0001.fits


In [86]:
t.tv(mask)

In [76]:
nomask=red.rd('field_image_no-mask_11985_rot90.0001.fits')

  Reading file: /home/holtz/raw/apo/jul22/UT220709/field_image_no-mask_11985_rot90.0001.fits


In [77]:
t.tv(nomask)


In [79]:
from pyvista import stars
s=stars.mark(t,rad=5,new=True)

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit
possible centroiding convergence issues, consider using a larger radius?


In [81]:
s

id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes32,float32
1,248.72,2398.21,59769.376375,180.0,,1.056
2,796.52,2418.34,59769.376375,180.0,,1.056
3,875.14,1265.74,59769.376375,180.0,,1.056
4,1450.27,1351.40,59769.376375,180.0,,1.056


In [82]:
maskcr=red.crrej(mask,crbox=[5,5])

  Iteration 0, zapping CRs with filter [5,5]...


In [99]:
from pyvista import slitmask
holes=slitmask.findholes(maskcr.data,thresh=4000,n=29)
t.tv(maskcr)
t.tvclear()
stars.mark(t,s,exit=True,color='r',rad=10)
stars.mark(t,holes,color='b',exit=True,rad=10)

/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/astropy/table/table.py:3488: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


y,x
float64,float64
2449.01183432,1009.24063116
2151.96449704,881.24260355
1859.01183432,921.236686391
2648.0,966.126482213
2539.0,1201.43083004
2234.0,727.126482213
2362.98811881,1163.24356436
1878.96421471,1522.47316103
2122.95418327,519.721115538


In [49]:
s2=stars.mark(t,rad=5)

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit
possible centroiding convergence issues, consider using a larger radius?


In [51]:
s2


id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes32,float32
1,248.71,2398.22,59769.376375,180.0,,1.056
2,804.73,2414.20,59769.376375,180.0,,1.056
3,1076.04,1258.60,59769.376375,180.0,,1.056
4,1450.27,1351.40,59769.376375,180.0,,1.056
5,248.72,2398.21,59769.376375,180.0,,1.056
6,799.19,2416.99,59769.376375,180.0,,1.056
7,1075.97,1258.34,59769.376375,180.0,,1.056
8,1450.27,1351.40,59769.376375,180.0,,1.056


In [52]:
stars.mark(t,s2[4:],color='r',rad=10)

/home/local/Anaconda3-2020.02/envs/py3.9/lib/python3.9/site-packages/astropy/table/table.py:3488: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  result = self.as_array() == other


Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit


id,x,y,MJD,EXPTIME,FILTER,AIRMASS
int32,float32,float32,float64,float32,bytes32,float32
5,248.72,2398.21,59769.376375,180.0,,1.056
6,799.19,2416.99,59769.376375,180.0,,1.056
7,1075.97,1258.34,59769.376375,180.0,,1.056
8,1450.27,1351.40,59769.376375,180.0,,1.056


In [84]:
stars.mark(t,holes,color='b',rad=10)

Hit c near desired star(s) to get centroid position
    i to use integer position of cursor
    n to get ID of nearest star
    q or e to quit
23
Star: 23 at (293.463668,2701.813149)
10
Star: 10 at (840.175214,2717.950855)
8
Star: 8 at (1106.415179,1547.924107)
3
Star: 3 at (1483.889625,1639.406181)


KeyError: 'id'

In [58]:
from astropy.table import vstack
h2=vstack([holes[17],holes[24],holes[11],holes[3]])

In [69]:
slitmask.fit(h2,s2[4:])

Rotation (degrees) :  0.7023411872010255
Translation (arcsec)  [ -9.37132715 -76.06268694]
[-729.95876289  655.51890034] [-774.26215265  351.70530317] [-775.28491211  350.21362305] 1.80863111138
[-183.82478632  669.9508547 ] [-228.3461122   372.83060663] [-224.80688477  368.99414062] 5.21963622216
[  82.41517857 -500.07589286] [  52.21587312 -793.84470437] [  51.9666748  -789.66027832] 4.19183982806
[ 459.88962472 -408.59381898] [ 428.54058509 -697.74247497] [ 426.27331543 -696.59875488] 2.53941082619
Rotation (degrees) :  0.7023411872010255
Translation (arcsec)  [ -9.45714522 -75.96874303]
Scale  0.996466664091
[-729.95876289  655.51890034] [-771.98739381  349.78504774] [-775.28491211  350.21362305] 3.32525242874
[-183.82478632  669.9508547 ] [-228.00025812  370.83570842] [-224.80688477  368.99414062] 3.68632680088
[  82.41517857 -500.07589286] [  51.57040747 -791.71734682] [  51.9666748  -789.66027832] 2.0948886844
[ 459.88962472 -408.59381898] [ 426.56543782 -695.95467887] [ 426.273

(0.7023411872010255, array([ -9.37132715, -76.06268694]))